In [28]:
import numpy as np
from typing import Tuple
from scipy.stats import mode
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd

csv_path = "diabetes.csv"

In [74]:
class KNNClassifier:
    k = 0
    test_split_ratio = 0
    x_train = 0
    y_train = 0
    x_test = 0
    y_test = 0
    counter = 0
    
    def __init__(self, k:int, test_split_ratio :float):
        self.k = k
        self.test_plit_ratio = test_split_ratio
        
        KNNClassifier.counter += 1

    @staticmethod
    def load_csv(csv_path:str)->Tuple[pd.core.frame.DataFrame, pd.core.frame.DataFrame]:
        dataset= pd.read_csv(csv_path)
        dataset = dataset.sample(frac=1, random_state=42)
        x,y = dataset.iloc[:,:-1],dataset.iloc[:,-1]
        return x,y
    
    @classmethod
    def train_test_split(self, features:pd.core.frame.DataFrame, labels:pd.core.frame.DataFrame):
        test_size = int(len(features)*  self.test_split_ratio)
        train_size = len(features) - test_size
        assert len(features) == test_size + train_size, "Size mismatch!"

        self.x_train, self.y_train = features.iloc[:train_size,:], labels.iloc[:train_size]
        self.x_test, self.y_test = features.iloc[train_size:train_size + test_size,:], labels.iloc[train_size:train_size + test_size]
        return test_size

In [86]:
a = KNNClassifier(3, 0.2)
x, y = a.load_csv("diabetes.csv")
b = a.train_test_split(x, y)

print(np.shape(x))
print(np.shape(y))
print()
print(np.shape(a.x_train))
print(np.shape(a.y_train))
print(np.shape(a.x_test))
print(np.shape(a.y_test))


TypeError: cannot unpack non-iterable NoneType object